In [551]:
import regex as re

In [552]:
def get_block_lengths(line: str) -> list[int]:
    
    blocks = [block for block in line.replace("?", ".").split(".") if len(block) > 0]
    block_lengths = [len(block) for block in blocks]
    return block_lengths

def satisfied(line: str, contiguous_broken: list[int]) -> bool:
    return get_block_lengths(line) == contiguous_broken


def still_possible(line: str, blocks: list[int]) -> bool:
    # print(line)
    num_hashes = line.count("#")
    num_question_marks = line.count("?")
    expected_num_hashes = sum(blocks)
    diff = expected_num_hashes - num_hashes
    if diff < 0: # more than expected:
        return False
    elif diff > 0:
        if diff > num_question_marks: # not enough to satisfy constraint
            return False

    current_line = line
    block_to_verify = 0

    while block_to_verify < len(blocks):
        match = re.match("^[^#\?]*(#+)", current_line)
        # print(match)
        if match:
            hashes = match.group(1)
            _, span_end = match.span()
            num_hashes = len(hashes)
            block = blocks[block_to_verify]

            # print(f"\t{match}", span_end, num_hashes, block)

            if num_hashes < block:
                return current_line[span_end] == "?"

            elif num_hashes > block:
                return False
            
            else: # num_hashes == block
                current_line = current_line[span_end:]
                block_to_verify += 1
        else:
            match = re.match("^[^#\?]*(#+)", current_line)
            return True
    return True

def constraint_broken(line: str, contiguous_broken: list[int]) -> bool:
    block_lengths = get_block_lengths(line)
    if "?" in line:
        return any([x > max(contiguous_broken) for x in block_lengths])
        # if ble:
        #     print(line, contiguous_broken, block_lengths)
            # return True
        # if len(block_lengths) > len(contiguous_broken):
        #     print(line, contiguous_broken, block_lengths)
        #     return True

        # for expected, actual in zip(contiguous_broken, block_lengths):
        #     if expected < actual:
        #         print(line, contiguous_broken, block_lengths)
        #         return True

        # return False
    else:
        return not satisfied(line, contiguous_broken)

In [553]:
def generate_solutions(line: str, contiguous_broken: list[int], current_block_index: int = 0) -> list[str]:
    if "?" in line:
        current_block = contiguous_broken[current_block_index]
        possible = still_possible(line, contiguous_broken)
        # print(line, possible)
        if possible:
            unbroken_solution = line.replace("?", ".", 1)
            broken_solution = line.replace("?", "#", 1)

            return generate_solutions(broken_solution, contiguous_broken) + generate_solutions(unbroken_solution, contiguous_broken)
        else:
            return []
    else:
        if satisfied(line, contiguous_broken):
            return [line]
        else:
            return []

In [554]:
generate_solutions(".??..??...?##.", [1,1,3])

['.#...#....###.', '.#....#...###.', '..#..#....###.', '..#...#...###.']

In [555]:
with open("12/example.txt") as f:
    lines = f.read().splitlines()
lines = [line.split() for line in lines]
line_constraints: list[tuple[str, list[int]]] = [(line, [int(x) for x in blocks.split(",")]) for line, blocks in lines]
line_constraints[:5]

[('???.###', [1, 1, 3]),
 ('.??..??...?##.', [1, 1, 3]),
 ('?#?#?#?#?#?#?#?', [1, 3, 1, 6]),
 ('????.#...#...', [4, 1, 1]),
 ('????.######..#####.', [1, 6, 5])]

In [556]:
from tqdm import tqdm

def get_solutions_for_lines(line_constraints):
    solutions = []
    for line, blocks in tqdm(line_constraints):
        possible_solutions = set(generate_solutions(line, blocks))
        solutions.append(possible_solutions)
    return solutions

solutions = get_solutions_for_lines(line_constraints)
num_solutions = [len(s) for s in solutions]
num_solutions

100%|██████████| 6/6 [00:00<00:00, 1643.86it/s]


[1, 4, 1, 1, 4, 10]

In [557]:
sum(num_solutions)

21

# Part 2

In [558]:
def expand_line(line: str, contiguous_broken: list[int], times=5) -> tuple[str, list[int]]:
    return "?".join([line] * times), contiguous_broken * times

In [559]:
expanded_constraints = [
    expand_line(*constraint, times=5)
    for constraint in line_constraints
]
expanded_constraints

[('???.###????.###????.###????.###????.###',
  [1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3]),
 ('.??..??...?##.?.??..??...?##.?.??..??...?##.?.??..??...?##.?.??..??...?##.',
  [1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3]),
 ('?#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#?',
  [1, 3, 1, 6, 1, 3, 1, 6, 1, 3, 1, 6, 1, 3, 1, 6, 1, 3, 1, 6]),
 ('????.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#...',
  [4, 1, 1, 4, 1, 1, 4, 1, 1, 4, 1, 1, 4, 1, 1]),
 ('????.######..#####.?????.######..#####.?????.######..#####.?????.######..#####.?????.######..#####.',
  [1, 6, 5, 1, 6, 5, 1, 6, 5, 1, 6, 5, 1, 6, 5]),
 ('?###??????????###??????????###??????????###??????????###????????',
  [3, 2, 1, 3, 2, 1, 3, 2, 1, 3, 2, 1, 3, 2, 1])]

In [560]:
solutions = get_solutions_for_lines(expanded_constraints)
num_solutions = [len(s) for s in solutions]
num_solutions

 17%|█▋        | 1/6 [00:04<00:24,  4.98s/it]


KeyboardInterrupt: 

In [561]:
line = ".###.?????????###??????????###??????????###??????????###????????"
blocks = [3, 2, 1, 3, 2, 1, 3, 2, 1, 3, 2, 1, 3, 2, 1]
still_possible(line, blocks), sum(blocks), line.count("#"), line.count("?")

(True, 30, 15, 47)

In [562]:
def possible_strings(line: str, block_length: int) -> list[str]:
    matches = []
    for match in re.finditer("[?#]{" + str(block_length) + "}", line, overlapped=True):
        span_start, span_end = match.span()

    re.match("[?#]{}", line)
line

'.###.?????????###??????????###??????????###??????????###????????'

In [578]:
def place_block(line: str, block: int) -> list[tuple[str, str]]:
    solutions = []
    pattern = "[.?][?#]{" + str(block) + "}[.?]"

    line_extended = f".{line}."
    for x in re.finditer(pattern, line_extended, overlapped=True):
        span_start, span_end = x.span()
        fixed, to_solve = line_extended[:span_start] + "." + ("#" * block) + ".", line_extended[span_end:-1]
        fixed = fixed.replace("?", ".")
        fixed = fixed[1:]
        solutions.append((fixed, to_solve))
    return solutions

def generate_solutions2(line: str, blocks: list[int]):
    solved = set()
    working = [("", line)]
    for block in blocks:
        new_working = []
        for (fixed, to_solve) in working:
            rest_solutions = place_block(to_solve, block)
            for (f, ts) in rest_solutions:
                sol = fixed + f + ts
                if satisfied(sol, blocks):
                    solved.add(sol.replace("?", "."))
                else:
                    if still_possible(sol, blocks):
                        new_working.append((fixed + f, ts))
        working = new_working
    return solved

def get_solutions_for_lines2(line_constraints):
    solutions = []
    for line, blocks in tqdm(line_constraints):
        possible_solutions = set(generate_solutions2(line, blocks))
        solutions.append(possible_solutions)
    return solutions

In [579]:
line, blocks = line_constraints[0]
line, blocks

('???.###', [1, 1, 3])

In [581]:
[len(generate_solutions2(*c)) for c in line_constraints]

[1, 4, 1, 1, 4, 10]